In [1]:
# !pip install ipynb # used to import another ipynb
# !pip install python-dotenv # used to load .env file

In [2]:
# Load env should run it very first
from dotenv import load_dotenv
load_dotenv()


True

In [11]:
# from preprocess.readcsv import read
# from preprocess import esClient
import eshelper
import csvhelper

import importlib
importlib.reload(eshelper)
importlib.reload(csvhelper)


# from ipynb.fs.full.preprocess.readcsv import read


<module 'csvhelper' from '/Users/roychiu/Desktop/ISS Master/GroupProject1/preprocess/csvhelper.py'>

In [4]:
# read recipe data
header = [
    "query",
    "recipe_id",
    "publisher",
    'source_url', 
    'image_url', 
    'social_rank', 
    'publisher_url', 
    'title', 
    'sum_cal', 
    'sum_fat', 
    'sum_carb', 
    'sum_protein'
]
recipeData = csvhelper.read("../data/recipe.csv", header)


In [6]:
#push recipe to elasticsearch
err = eshelper.bulkIndex(recipeData,"recipes","{recipe_id}")
if err != None:
    print(err)
else:
    print("pushed to elasticsearch")

pushing 3166 documents
pushed to elasticsearch


In [7]:
# read ingredient data
header = [
    "query",
    "recipe_id",
    "quantity",
    "unit",
    "weight_g",
    "ingredient",
    "nutrition_key",
    "nutrition_value"
    "cal",
    'fat', 
    'cah', 
    'protein', 
    'original_ingredient'
]
ingredientData = csvhelper.read("../data/ingredient.csv", header)

In [8]:
# create dictionary by recipe id
recipeDict = {}
for recipe in recipeData:
    recipe_id = recipe["recipe_id"]
    recipeDict[recipe_id] = recipe.copy()
    recipeDict[recipe_id]["ingredients"] = []
    recipeDict[recipe_id]["ingredients_weight_g"] = []
    recipeDict[recipe_id]["total_ingredient_weight_g"] = 0
    recipeDict[recipe_id]["min_ingredient_weight_g"] = float("inf")

# push ingredient to recipe
for ingredient in ingredientData:
    recipe_id = ingredient["recipe_id"]
    recipeDict[recipe_id]["ingredients"].append(ingredient)
    recipeDict[recipe_id]["total_ingredient_weight_g"] += ingredient["weight_g"]
    if ingredient["weight_g"] < recipeDict[recipe_id]["min_ingredient_weight_g"]:
        # get the min weight and normalise it later
        recipeDict[recipe_id]["min_ingredient_weight_g"] = ingredient["weight_g"]

# normalise ingredient
for recipe_id in recipeDict:
    for ingredient in recipeDict[recipe_id]["ingredients"]:
        times = int(ingredient['weight_g']/recipeDict[recipe_id]["min_ingredient_weight_g"])
        for i in range(times):
            recipeDict[recipe_id]["ingredients_weight_g"].append(ingredient["ingredient"])

In [9]:
#push recipe detail to elasticsearch
recipes = []
for key in recipeDict:
    r = recipeDict[key].copy()
    del r["ingredients"]
    recipes.append(r)
err = eshelper.bulkIndex(recipes,"recipe_detail","{recipe_id}")
if err != None:
    print(err)
else:
    print("pushed to elasticsearch")

pushing 2256 documents
pushed to elasticsearch


In [12]:
#search documents
eshelper.searchByIngredient("recipe_detail*", "cheese", 10)

Got 10 Hits:
